## <center> <h1> CS 401: Natural Language Processing</h1></center>
### <center> <h1> Project 4 </h1></center>
#### <center> <h1> Steven & Anamay </h1></center>

## Question 1

In [1]:
import numpy
import warnings
numpy.set_printoptions(precision=3)
def Viterbi(A, B, x):
    '''Viterbi decoding algorithm (logarithmic probabilities)
       A is the transition probability matrix (includes start state 0)
       B is the emission probability matrix (includes 0's for state 0)
         - stored as a dictionary of (state,symbol) pairs
       x is the sequence of observations'''
       
    n = len(x)                                        # number of observations (t)
    m = len(A)                                        # number of states (including start state)
    v = numpy.zeros((m, n + 1))  - numpy.inf          # matrix of v[q,i] values, initialized to -infinity
    v[0, 0] = 0                                       # log prob of starting in the start state is 0
    s = numpy.zeros((m, n + 1), dtype = numpy.int32)  # s[q,i] = optimal previous state for v[q,i]

    ####################################################################################
    # Fill in the tables (v and s), column by column, according to the recurrence, HERE.
    for i in range(1,n+1): # words
        for j in range(m): # states
            lister = []
            for k in range(m):
                value = v[k,i-1]+A[k,j]+B[j,x[i-1]]
                #print("Previous log prob:",v[k,i-1])
                #print("A value:",A[k,j])
                #print("B value",B[j,x[i-1]],"i:",i,"j:",j,"k:",k)
                lister.append(value)
            v[j,i] = max(lister)
            s[j,i]=lister.index(max(lister))
    ####################################################################################

    maxS = numpy.argmax(v[:,n])  # index (state) of the maximum value in the last column of v
    maxP = v[maxS, n]            # the maximum value in the last column of v

    # Construct the optimal sequence of states by backtracking.

    path = numpy.zeros(n + 1, dtype = numpy.int32)  # init state sequence to be a list of zeroes
    path[n] = maxS                                  # start in state maxS
    for i in range(n - 1, -1, -1):                  # and follow the "links" backwards
        path[i] = s[path[i + 1], i + 1]
        
    #print('maxS = ', maxS)
    #print('v =\n', v)
    #print('s =\n', s)
    return maxP, path

In [2]:
warnings.filterwarnings('ignore')
def main():
    stateNames = ['<s>', 'NNP', 'MD', 'VB', 'JJ', 'NN', 'RB', 'DT']
    
    #                  <s>   NNP     MD      VB      JJ      NN      RB      DT
    A = numpy.matrix([[0.0, 0.2767, 0.0006, 0.0031, 0.0453, 0.0449, 0.0510, 0.2026],  # <s> start
                      [0.0, 0.3777, 0.0110, 0.0009, 0.0084, 0.0584, 0.0090, 0.0025],  # NNP
                      [0.0, 0.0008, 0.0002, 0.7968, 0.0005, 0.0008, 0.1698, 0.0041],  # MD
                      [0.0, 0.0322, 0.0005, 0.0050, 0.0837, 0.0615, 0.0514, 0.2231],  # VB
                      [0.0, 0.0366, 0.0004, 0.0001, 0.0733, 0.4509, 0.0036, 0.0036],  # JJ
                      [0.0, 0.0096, 0.0176, 0.0014, 0.0086, 0.1216, 0.0177, 0.0068],  # NN
                      [0.0, 0.0068, 0.0102, 0.1011, 0.1012, 0.0120, 0.0728, 0.0479],  # RB
                      [0.0, 0.1147, 0.0021, 0.0002, 0.2157, 0.4744, 0.0102, 0.0017]]) # DT    
    Alog = numpy.log(A)
                      
    B = {(0, 'Janet'): 0.0, (0, 'will'): 0.0, (0, 'back'): 0.0, (0, 'the'): 0.000048, (0, 'bill'): 0.0,
         (1, 'Janet'): 0.000032, (1, 'will'): 0.0, (1, 'back'): 0.0, (1, 'the'): 0.000048, (1, 'bill'): 0.0,
         (2, 'Janet'): 0.0, (2, 'will'): 0.308431, (2, 'back'): 0.0, (2, 'the'): 0.0, (2, 'bill'): 0.0,
         (3, 'Janet'): 0.0, (3, 'will'): 0.000028, (3, 'back'): 0.000672, (3, 'the'): 0.0, (3, 'bill'): 0.000028,
         (4, 'Janet'): 0.0, (4, 'will'): 0.0, (4, 'back'): 0.000340, (4, 'the'): 0.0, (4, 'bill'): 0.0,
         (5, 'Janet'): 0.0, (5, 'will'): 0.000200, (5, 'back'): 0.000223, (5, 'the'): 0.0, (5, 'bill'): 0.002337,
         (6, 'Janet'): 0.0, (6, 'will'): 0.0, (6, 'back'): 0.010446, (6, 'the'): 0.0, (6, 'bill'): 0.0,
         (7, 'Janet'): 0.0, (7, 'will'): 0.0, (7, 'back'): 0.0, (7, 'the'): 0.506099, (7, 'bill'): 0.0}
    
    Blog = {(s, t): numpy.log(B[(s,t)]) for (s, t) in B}
         
    words = ['Janet', 'will', 'back', 'the', 'bill']
    
    prob, path = Viterbi(Alog, Blog, words)
    print("Probability:",prob)
    for i in range(len(words)):
        print(words[i], stateNames[path[i+1]])
    
main()

Probability: -33.8388667762
Janet NNP
will MD
back VB
the DT
bill NN


## Question 2 

In [3]:
import time
import nltk
from nltk.corpus import brown
import numpy as np
#nltk.download('universal_tagset')

### Creating training and testing sets

In [4]:
taggedWords = brown.tagged_words(tagset='universal')
Fulldata = [(word,tag) for (word,tag) in taggedWords]
from sklearn.model_selection import train_test_split
train, test = train_test_split(Fulldata, test_size=0.2,random_state=101) # setting seed 
print(len(train)/(len(train)+len(test))*100) # approx. 80-20 split

79.99994832895851


### Getting tags (states)

In [5]:
tag_fd = nltk.FreqDist(tag for (word, tag) in train) # contains unique tags with their frequencies
alltags = {}
index=1
for pos in tag_fd:
    alltags[pos] = index
    index+=1
alltags['<s>'] = 0

In [6]:
for tag in alltags:
    print(tag,alltags[tag])

NOUN 1
VERB 2
ADP 3
. 4
DET 5
ADJ 6
CONJ 7
PRT 8
NUM 9
PRON 10
ADV 11
X 12
<s> 0


### Initial probabilities

In [7]:
x = time.time()
init = {}
see = brown.tagged_sents(tagset='universal')
Fullsent = [sent for sent in see]
trainsent, testsent = train_test_split(Fullsent, test_size=0.2,random_state=101) # using same seed 
for sent in trainsent:
    if ('<s>',sent[0][1]) not in init:
        init[('<s>',sent[0][1])] = 0
    init[('<s>',sent[0][1])] +=1
    init[(sent[0][1],'<s>')] = 0
init[('<s>','<s>')]=0
for each in init:
    init[each] = init[each]/len(trainsent)
print("Run-time: ",time.time()-x,"seconds")

Run-time:  5.346083164215088 seconds


### Constructing transition probability matrix

In [8]:
x = time.time()
bigramsFreq = {}
allTags = [tag for (word, tag) in train]
bigrams = nltk.ngrams(allTags, 2) # getting bigrams
bigramsFreq = dict(nltk.FreqDist(bigrams))
bigramsFreq2 = {**bigramsFreq}
for each in bigramsFreq2:
    for other in bigramsFreq2:
        if (each[0],other[1]) not in bigramsFreq2: # need to get all combinations of states
            bigramsFreq[each[0],other[1]]=0
del bigramsFreq2
print("Run-time: ",time.time()-x,"seconds")

Run-time:  1.7405118942260742 seconds


In [9]:
for pos in tag_fd:
    for each in bigramsFreq:
        if each[0]==pos:
            bigramsFreq[each]=bigramsFreq[each]/tag_fd[pos]
AllBigrams = {**bigramsFreq, **init}
print(len(AllBigrams)) # 13x13 matrix

169


In [10]:
AllBigrams2 = {**AllBigrams} # deep-copying dictionary
for first,second in AllBigrams2:
    AllBigrams[(alltags[first],alltags[second])] = AllBigrams[(first,second)]
    del AllBigrams[(first,second)]
del AllBigrams2

In [11]:
A = np.zeros((len(alltags), len(alltags))) # square matrix
A = np.matrix(A)
for first,second in AllBigrams:
    A[first,second] = AllBigrams[(first,second)]

In [12]:
A

matrix([[ 0.   ,  0.141,  0.045,  0.123,  0.089,  0.212,  0.035,  0.049,
          0.037,  0.018,  0.161,  0.091,  0.   ],
        [ 0.   ,  0.237,  0.158,  0.125,  0.127,  0.118,  0.072,  0.032,
          0.026,  0.013,  0.043,  0.048,  0.001],
        [ 0.   ,  0.237,  0.158,  0.124,  0.127,  0.118,  0.072,  0.032,
          0.026,  0.013,  0.042,  0.049,  0.001],
        [ 0.   ,  0.239,  0.158,  0.123,  0.128,  0.117,  0.072,  0.034,
          0.026,  0.013,  0.042,  0.048,  0.001],
        [ 0.   ,  0.238,  0.157,  0.124,  0.128,  0.117,  0.072,  0.033,
          0.025,  0.013,  0.043,  0.048,  0.001],
        [ 0.   ,  0.235,  0.158,  0.124,  0.126,  0.118,  0.074,  0.033,
          0.025,  0.014,  0.042,  0.049,  0.001],
        [ 0.   ,  0.238,  0.156,  0.125,  0.128,  0.118,  0.072,  0.033,
          0.026,  0.013,  0.043,  0.048,  0.001],
        [ 0.   ,  0.232,  0.159,  0.127,  0.126,  0.118,  0.073,  0.035,
          0.026,  0.012,  0.042,  0.05 ,  0.001],
        [ 0.   ,

### Constructing emission probability matrix

In [13]:
# Initializing B matrix with 0s
x = time.time()
B = {}
for tag in alltags:
    for w,t in train:
        B[alltags[tag],w]=0
print("Runtime (in seconds):",time.time()-x)

Runtime (in seconds): 8.606604814529419


In [14]:
# For start of sentence word emission probabilities
for sent in trainsent:
    if (0,sent[0][0]) in B:
        B[0,sent[0][0]]+=1
for each in B:
    if each[0]==0:
        B[each] = B[each]/len(trainsent)
# For all other tags' word emission probabilities
word_tag = dict(nltk.FreqDist((word,tag) for (word, tag) in train))
for word,tag in word_tag:
    B[alltags[tag],word]=word_tag[word,tag]/tag_fd[tag]

In [15]:
B

{(1, 'Americans'): 0.0003401915959068147,
 (1, 'is'): 0,
 (1, 'whites'): 5.896654329051455e-05,
 (1, 'Bosphorus'): 2.267943972712098e-05,
 (1, 'by'): 0,
 (1, ','): 0,
 (1, 'mind'): 0.0010024312359387473,
 (1, 'the'): 0,
 (1, 'small'): 4.5358879454241965e-06,
 (1, 'miles'): 0.0006032730967414181,
 (1, 'with'): 0,
 (1, 'This'): 0,
 (1, 'analyzable'): 0,
 (1, 'And'): 0,
 (1, 'at'): 0,
 (1, 'All'): 0,
 (1, 'Chicago'): 0.00033565570796139055,
 (1, 'four'): 0,
 (1, 'was'): 0,
 (1, 'The'): 0,
 (1, 'two'): 0,
 (1, 'unless'): 0,
 (1, 'complex'): 0.0001179330865810291,
 (1, 'among'): 0,
 (1, 'I'): 1.8143551781696786e-05,
 (1, 'room'): 0.001365302271572683,
 (1, 'Molotov'): 1.8143551781696786e-05,
 (1, 'are'): 0,
 (1, '$8.50'): 4.5358879454241965e-06,
 (1, ')'): 0,
 (1, 'Barton'): 8.164598301763554e-05,
 (1, 'this'): 0,
 (1, 'a'): 0,
 (1, 'use'): 0.0012836562885550475,
 (1, 'direction'): 0.0004671964583786922,
 (1, 'of'): 0,
 (1, 'they'): 0,
 (1, 'former'): 0,
 (1, 'and'): 0,
 (1, 'effete'): 0,
 

## Question 3

In [16]:
import random
def substitute(sent,B):
    """
        Helper function to replace unknown words in the
        testset with the most likely tag for it based on
        word structure.
        
        Parameters:
              sent: sentence with words to be tagged
                 B: emission matrix to check if word in 
                    sentence is not in the training set.
      
        Return value: sentence with adjusted words
    """
    lister = []
    sent2 = []
    for key in B:
        lister.append(key[1])
    lister = list(set(lister))
    for word in sent:
        if word not in lister:
            if word[0].isupper() or word[-1]=='s' or word[-2:]=="'s": # possible noun endings
                word = tag_look(1,B) # noun tag
            elif word[-2:]=="ed" or word[-3:]=="ing": # possible verb endings
                word = tag_look(2,B) # verb tag
            elif word[-2:]=="ly": # possible adverb ending
                word = tag_look(11,B) # adverb tag
            else:
                index = random.randint(1,12) # picks a random tag to assign to a
                                             # word (more variance than the max. 'unk' tag)
                word = tag_look(index,B) # else just pick a random tag
        sent2.append(word)
    return sent2

In [17]:
def tag_look(i,Blog):
    """
        Helper function to find the most probable word
        within a given tag category
        
        Parameter:
                i: index of tag
             Blog: Emission matrix   
        Return value: Most common word to appear in tag 'i'
    """
    tups = []
    for key in Blog:
        if key[0] == i:
            tups.append((key[1],Blog[key]))
    tups.sort(reverse=True, key=lambda tup: tup[1])
    return tups[0][0]

In [18]:
warnings.filterwarnings('ignore')
stateNames = ['<s>', 'NOUN', 'VERB', 'ADP', '.', 'DET', 'ADJ', 'CONJ', 'PRT', 'NUM', 'PRON', 'ADV', 'X']
Alog = numpy.log(A)
Blog = {(s, t): numpy.log(B[(s,t)]) for (s, t) in B}
x= time.time()
TP = 0
Total = 0
for sentTags in testsent: # Takes approximately an hour to run
    sent = [x[0] for x in sentTags]
    tags = [x[1] for x in sentTags]
    sent2 = substitute(sent,Blog)   # helper function call to assign probabilities to known words in sentence
    prob, path = Viterbi(Alog, Blog, sent2)
    for i in range(len(sent)):
        if (stateNames[path[i+1]]==tags[i]):
            TP += 1 # True positives
        Total +=1   # Total words
print("Time taken: ",time.time()-x) 

Time taken:  17.344727993011475


In [19]:
print("Accuracy:",round(TP/Total*100,2),"%")

Accuracy: 95.72 %


Overall accuracy of 94.97% on the entire test set.

## Question 4

In our HMM tagger's current state, our most frequent errors are from unknown words that were not tagged by our logic ('substitute' function) that determines part of speech based on the morphology of the word ( -ing, -ed are verbs, -'s is a noun, -ly is an adverb). If we cannot find a tag for the unknown word based on these general rules for POS identification, we randomly select a class. We do this because we already don't know the correct tag so this only has the potential to improve the accuracy (1/12 chance of being randomly correct). We figured this was better than introducing an 'unknown' word frequency for each tag and always picking the highest probability tag for 'unknown'. With our approach,we have an accuracy rate of about 94.97% for the entire test data. To improve our accuracy we could add even more rules for manually determining the POS of an unknown word (maybe context-free grammars would help create such rules). 